## 텍스트를 숫자로 바꾸는 텍스트 임베딩   
    RAG의 기본 작동 과정은 아래와 같다.   
    # Document(문서) -> Chunk(글 덩어리) -> Vector Store -- 질문 임베딩과 고 유사도 Vector 청크 프롬프트 투입-- -> LLM(토큰 제한)   
    이런 작동 단계에서 임베딩은 1. 문서 청크를 벡터 DB에 저장할 때 2. 사용자의 질문에 답할 근거를 벡터 DB에서 검색할 때 사용된다.   
    - 사용자 질문에 대한 근거를 문서 내 키워드 검색을 통해서 수행할 수 없나?   
      - 가능하지만 고려해야 할 제약이 많다.   
        1. 키워드는 무엇인가   
        2. 사용자 질문이 키워드 몇 개로 대표되나   
        3. 질문 속에 근거를 찾기 위한 키워드가 포함되어 있나   
    - 문서를 분할한 청크를 임베딩으로 변환하여 사용자 질문과 높은 유사도를 지닌 청크를 찾는게 더 효율적   

    임베딩 모델이란?
        - 텍스트를 수치로 변환하는 작업이 임베딩 / 대량의 텍스트로 사전 학습된 모델을 활용함 / 대표 모델은 BERT   
        - 대량의 텍스트 문서를 레이블링 하지 않고 encoder 구조를 활용해 사전학습함으로 Sentence-BERT모델은 지정된 max_token값 범위 안에서 문장을 벡터화하여 학습   
        - 단어와 문장의 맥락 정보까지 모델이 파악해서 I ate an apple이라는 문장 속의 apple이 스마트폰 회사나 스마트폰이 아니라는것도 파악 가능 / 이런 패턴과 맥락을 수치로 나타냄   
    # Open model은 허깅페이스에서 무료로 사용할 수 있고 Closed model은 기업의 모델로 API를 호출해서 사용한다.   

In [3]:
# OpenAI 임베딩 모델 대신 구글 임베딩 모델로 오픈소스 모델 활용 예제 

import os
from langchain_google_genai import GoogleGenerativeAIEmbeddings

api_key = os.environ["GEMINI_API_KEY"]

embedding_model = GoogleGenerativeAIEmbeddings(model="models/embedding-001", google_api_key=api_key)
embeddings = embedding_model.embed_documents(
    [
        "Hi there!",
        "Oh, hello!",
        "What's your name?",
        "My friends call me World.",
        "Hello, World!"
    ]
)

len(embeddings), len(embeddings[0])

(5, 768)

OpenAI 임베딩 모델은 한 문장에 대한 임베딩 값을 1536개 벡터로 변환 / 구글임베딩 모델은 768

In [5]:
from langchain.document_loaders import PyPDFium2Loader
from langchain_text_splitters import RecursiveCharacterTextSplitter

import warnings 
warnings.filterwarnings("ignore")

# 임베딩 모델 API 호출
embedding_model = GoogleGenerativeAIEmbeddings(model="models/embedding-001", google_api_key=api_key)

# PDF 문서 로드
loader = PyPDFium2Loader("./data/[이슈리포트 2022-2호] 혁신성장 정책금융 동향.pdf")
pages = loader.load()

# PDF문서를 여러 청크로 분할
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=100
)

texts = text_splitter.split_documents(pages)

# 임베딩 모델로 청크들 임베딩 변환
embeddings = embedding_model.embed_documents([i.page_content for i in texts])
len(embeddings), len(embeddings[0])

(53, 768)

이렇게 벡터로 바꾸고 각 벡터 간의 거리를 측정해서 거리가 가까운 순서대로 유사도를 계산한다.    
임베딩 간의 거리를 구할 때 코사인 유사도를 활용한다.

In [6]:
# 텍스트들 임베딩 벡터 간 유사도 측정

examples = embedding_model.embed_documents(
    [
        "안녕하세요.",
        "제 이름은 홍두깨입니다.",
        "이름이 무엇인가요?",
        "랭체인은 유용합니다."
    ]
)

# 예시 질문과 답변 임베딩 
embedded_query_q = embedding_model.embed_query("이 대화에서 언급된 이름은 무엇입니까?")
embedded_query_a = embedding_model.embed_query("이 대화에서 언급된 이름은 홍길동입니다.")

# 대화에서 언급된 이름에 대한 질문에 가장 적절한 답을 찾아야 하는 상황 가정
# 예시 답변으로 주어진 답변 문장과 질문 유사도가 가장 높은 문장은 두번째와 세번째 순으로 유사도가 높게 나올 것으로 예상

from numpy import dot
from numpy.linalg import norm 
import numpy as np 
 
def cos_sim(A, B):
    return dot(A, B)/(norm(A)*norm(B))

print(cos_sim(embedded_query_q, embedded_query_a))
print(cos_sim(embedded_query_a, examples[1]))
print(cos_sim(embedded_query_a, examples[3]))

0.9430132914597402
0.7049958828721193
0.6887223978447613


In [11]:
# 텍스트들 임베딩 벡터 간 유사도 측정

examples = embedding_model.embed_documents(
    [
        "Hi.",
        "My name is Tim.",
        "What's your name?",
        "Langchain is useful."
    ]
)

# 예시 질문과 답변 임베딩 
embedded_query_q = embedding_model.embed_query("What's the name mentioned in the conversation?")
embedded_query_a = embedding_model.embed_query("The name mentioned in the conversation is Ted.")

# 대화에서 언급된 이름에 대한 질문에 가장 적절한 답을 찾아야 하는 상황 가정
# 예시 답변으로 주어진 답변 문장과 질문 유사도가 가장 높은 문장은 두번째와 세번째 순으로 유사도가 높게 나올 것으로 예상

from numpy import dot
from numpy.linalg import norm 
import numpy as np 
 
def cos_sim(A, B):
    return dot(A, B)/(norm(A)*norm(B))

print(cos_sim(embedded_query_q, embedded_query_a))
print(cos_sim(embedded_query_a, examples[1]))
print(cos_sim(embedded_query_a, examples[3]))

0.8385674802306704
0.6190615117812432
0.5094754342820353


In [13]:
# Open source 임베딩 모델 유사도 측정 
# 1. jhgan/ko-sroberta-multitask 임베딩 모델

from langchain_community.embeddings import HuggingFaceEmbeddings

# HuggingFaceEmbedding 함수로 오픈 소스 임베딩 모델 로드
model_name = "jhgan/ko-sroberta-multitask"
ko_embeddings = HuggingFaceEmbeddings(model_name=model_name)

examples = ko_embeddings.embed_documents(
    [
        "안녕하세요.",
        "제 이름은 홍두깨입니다.",
        "이름이 무엇인가요?",
        "랭체인은 유용합니다."
    ]
)

embedded_query_q = ko_embeddings.embed_query("이 대화에서 언급된 이름은 무엇입니까?")
embedded_query_a = ko_embeddings.embed_query("이 대화에서 언급된 이름은 홍길동입니다.")

print(cos_sim(embedded_query_q, embedded_query_a))
print(cos_sim(embedded_query_a, examples[1]))           
print(cos_sim(embedded_query_a, examples[3]))

0.6070005807182443
0.3590359241854671
0.2546047637215995


In [12]:
# 2. BAAI/bge-small-en 임베딩 모델 활용

from langchain_community.embeddings import HuggingFaceEmbeddings

model_name = "BAAI/bge-small-en"
bge_embeddings = HuggingFaceEmbeddings(model_name=model_name)

examples = bge_embeddings.embed_documents(
    [
        "안녕하세요.",
        "제 이름은 홍두깨입니다.",
        "이름이 무엇인가요?",
        "랭체인은 유용합니다."
    ]
)

embedded_query_q = bge_embeddings.embed_query("이 대화에서 언급된 이름은 무엇입니까?")
embedded_query_a = bge_embeddings.embed_query("이 대화에서 언급된 이름은 홍길동입니다.")

print(cos_sim(embedded_query_q, embedded_query_a))
print(cos_sim(embedded_query_a, examples[1]))
print(cos_sim(embedded_query_a, examples[3]))

0.9554541293280951
0.9322697037415678
0.9105264657723084


유사도 측정 시 1번 모델에 비해 2번 모델의 값이 크다.    
이유는 해당 임베딩 모델은 사전 학습 시 영어와 중국어에 대해서만 학습되었기 때문에 다국어 모델인 text-embedding-3-small에 비해 한글 문장 임베딩 성능이 떨어진다.   
이처럼 임베딩 모델은 사전 학습 시 어떤 언어 데이터셋으로 학습되었는지도 특정 언어의 문장 임베딩 성능에 영향을 준다.   
RAG의 청크 임베딩 과정에서 어떤 언어를 사용하는지 고려하여 적절한 임베딩 모델을 사용하는 것이 중요하다. 

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer

corpus = [
    "This is the first draft.",
    "This is the second one.",
    "Where is the third one?",
    "Is this the first draft?"
]

vectorizer = TfidfVectorizer()
vectors = vectorizer.fit_transform(corpus)

from numpy import dot
from numpy.linalg import norm 

def cos_sim(A, B):
    A = A.toarray().flatten()
    B = B.toarray().flatten()
    return dot(A, B)/(norm(A)*norm(B))

print(cos_sim(vectors[0], vectors[1]))
print(cos_sim(vectors[1], vectors[2]))
print(cos_sim(vectors[0], vectors[2]))
print(cos_sim(vectors[0], vectors[3]))

0.4005386104557453
0.4085441476443125
0.20658353143916394
1.0000000000000002


In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

corpus = [
    "This is the first draft.",
    "This is the second one.",
    "Where is the third one?",
    "Is this the first draft?"
]

vectorizer = TfidfVectorizer()
vectors = vectorizer.fit_transform(corpus)

print(cosine_similarity(vectors[0], vectors[1])[0][0])
print(cosine_similarity(vectors[1], vectors[2])[0][0])
print(cosine_similarity(vectors[0], vectors[2])[0][0])
print(cosine_similarity(vectors[0], vectors[3])[0][0])

0.40053861045574535
0.4085441476443126
0.20658353143916397
1.0000000000000002


In [1]:
import numpy as np
from transformers import AutoTokenizer, AutoModel
import torch
from sklearn.metrics.pairwise import cosine_similarity

import warnings 
warnings.filterwarnings("ignore")

def get_sentence_embeddings(sentences, model_name='sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2'):
    # 토크나이저와 모델 로드
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModel.from_pretrained(model_name)
    # 연산 중에 경사 계산을 비활성화
    embeddings = []
    
    with torch.no_grad():
        for sentence in sentences:
            # 토큰화 및 모델 입력 준비
            inputs = tokenizer(sentence, padding=True, truncation=True, return_tensors="pt")
            # 모델을 통한 임베딩 계산
            outputs = model(**inputs)
            # 마지막 은닉 상태 사용
            embeddings.append(outputs.last_hidden_state.mean(dim=1).squeeze().numpy())
    
    return np.array(embeddings)

def compute_similarity_matrix(sentences):
    # 문장 임베딩 구하기
    embeddings = get_sentence_embeddings(sentences)
    # 유사도 행렬 계산
    similarity_matrix = cosine_similarity(embeddings)
    
    return similarity_matrix

def compare_sentences(sentence1, sentence2):
    sentences = [sentence1, sentence2]
    similarity_matrix = compute_similarity_matrix(sentences)
    
    return similarity_matrix[0, 1]

In [2]:
# 예제 사용

# 예제 문장들
sentences = [
    "나는 학교에 갑니다",
    "학교에 나는 갑니다",  # 단어 순서만 바뀐 문장
    "나는 학교로 향합니다",  # 유사한 의미를 가진 문장
    "학교는 집에서 멀리 있습니다"  # 관련은 있지만 다른 의미의 문장
]
    
# 문장 쌍 간의 유사도 계산 및 출력
for i in range(len(sentences)):
    for j in range(i+1, len(sentences)):
        similarity = compare_sentences(sentences[i], sentences[j])
        print(f"문장 1: '{sentences[i]}'")
        print(f"문장 2: '{sentences[j]}'")
        print(f"유사도: {similarity:.4f}\n")
            
# 전체 유사도 행렬 계산
similarity_matrix = compute_similarity_matrix(sentences)
print("전체 유사도 행렬:")
print(similarity_matrix)
    
# 단어 순서가 중요한 예제
order_examples = [
    "영희가 철수를 좋아한다",
    "철수가 영희를 좋아한다",  # 단어 순서가 바뀌어 의미가 달라짐
    "영희는 철수에게 선물을 주었다",
    "철수는 영희에게 선물을 주었다"  # 단어 순서가 바뀌어 의미가 달라짐
]
    
# 단어 순서가 중요한 예제의 유사도 행렬
order_matrix = compute_similarity_matrix(order_examples)
print("\n단어 순서가 중요한 문장들의 유사도 행렬:")
for i, sentence in enumerate(order_examples):
    print(f"{i}: {sentence}")
print(order_matrix)

문장 1: '나는 학교에 갑니다'
문장 2: '학교에 나는 갑니다'
유사도: 0.9965

문장 1: '나는 학교에 갑니다'
문장 2: '나는 학교로 향합니다'
유사도: 0.9121

문장 1: '나는 학교에 갑니다'
문장 2: '학교는 집에서 멀리 있습니다'
유사도: 0.4730

문장 1: '학교에 나는 갑니다'
문장 2: '나는 학교로 향합니다'
유사도: 0.8995

문장 1: '학교에 나는 갑니다'
문장 2: '학교는 집에서 멀리 있습니다'
유사도: 0.4709

문장 1: '나는 학교로 향합니다'
문장 2: '학교는 집에서 멀리 있습니다'
유사도: 0.5149

전체 유사도 행렬:
[[0.99999964 0.9965155  0.91214895 0.47301492]
 [0.9965155  0.99999994 0.89945614 0.47087014]
 [0.91214895 0.89945614 0.9999998  0.51485926]
 [0.47301492 0.47087014 0.51485926 0.99999994]]

단어 순서가 중요한 문장들의 유사도 행렬:
0: 영희가 철수를 좋아한다
1: 철수가 영희를 좋아한다
2: 영희는 철수에게 선물을 주었다
3: 철수는 영희에게 선물을 주었다
[[1.         0.9836133  0.9531579  0.96602637]
 [0.9836133  1.0000004  0.94147426 0.96708775]
 [0.9531579  0.94147426 1.         0.98728764]
 [0.96602637 0.96708775 0.98728764 1.        ]]
